# ETL - ARCHIVOS COMPLEMENTARIOS

### Archivos complementarios --> Historical Weather API 

In [1]:
import requests
import pandas as pd

base_url = 'https://archive-api.open-meteo.com/v1/archive?'
#Parametros de datos, DEFINIR RANGO TEMPORAL
params = {
    'latitude'  : '40.7143',
    'longitude' : '74.006',
    'start_date': '2013-01-01',
    'end_date'  : '2023-07-31',
    'daily'     : 'temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,precipitation_sum',
    'timezone' :  'auto'
}

response = requests.get(base_url, params=params)

if response.status_code == 200:
    data_json = response.json()
    print('Solicitud completada, datos climaticos obtenidos')
else:
    print('Error al realizar la solicitud:', response.status_code)

Solicitud completada, datos climaticos obtenidos


In [2]:
daily_df = pd.DataFrame(data_json["daily"])

In [3]:
daily_df['time'] = pd.to_datetime(daily_df['time'])

In [65]:
#daily_df['time'] = daily_df['time'].apply( lambda x: x.strftime("%d-%m-%Y"))


In [4]:
daily_df.to_csv('./Complementary//weather.csv', index=False)

### Archivos complementarios --> Taxi trip yellow

In [45]:
# import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By

# Inicializa el navegador web (puede requerir la descarga de un controlador específico, como chromedriver)
driver = webdriver.Chrome()

# URL de la página web
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
driver.get(url)

# Expando lista de archivos parquet
expand_link = driver.find_element( By.XPATH , '/html/body/div[1]/div/div/div[1]/div/div[2]/div/div[1]/div/div[2]/div/div/div/div[2]/div/div/div[1]/div/a[1]')
expand_link.click()

# Espera a que se cargue el contenido después del clic
driver.implicitly_wait(1)

# Encuentra todos los elementos <a> con la clase "exitlink"
exit_links = driver.find_elements(By.CLASS_NAME, 'exitlink')

# Itera sobre los elementos y extrae los enlaces
yellow_taxis = []
green_taxis = []
for link in exit_links:
    file_url = link.get_attribute("href")
    file_title = link.get_attribute("title")

    if file_title == 'Yellow Taxi Trip Records':
        yellow_taxis.append(file_url)
    elif file_title == 'Green Taxi Trip Records':
        green_taxis.append(file_url)

driver.quit()

Se redujo el tamaño temporal, que es el último año. Por tamaño de archivos se reverá cargar y eliminar a la par los archivos.

In [43]:
import re
import os
import requests
from urllib.request import urlretrieve


regex = r"(\d{4})-(\d{2})\.parquet$" #Expresión regular para extrer el año

path_yellow = './Complementary/yellow_taxis'
if not os.path.exists(path_yellow):
    os.makedirs(path_yellow)

# Filtra los archivos por el año en el rango deseado, en este caso del ultimo año, mayo a mayo
for archivo in yellow_taxis:
    match = re.search(regex, archivo)
    if match:
        year = int(match.group(1))
        month = int(match.group(2))
        if (year == 2022 and month >= 5) or (year == 2023 and month <= 5):
            name_file = os.path.basename(archivo)
            path = os.path.join(path_yellow, name_file)
            urlretrieve(archivo, path)

Esto se redujo el tiempo 

In [35]:
path_green = './Complementary/green_taxis'
if not os.path.exists(path_green):
    os.makedirs(path_green)

for archivo in green_taxis:
    match = re.search(regex, archivo)
    if match:
        year = int(match.group(1))
        month = int(match.group(2))
        if (year == 2022 and month >= 5) or (year == 2023 and month <= 5):
            name_file = os.path.basename(archivo)
            path = os.path.join(path_yellow, name_file)
            urlretrieve(archivo, path)

77

Trabajando ahora con los archivos. Todo esto podemos juntarlo en un script para que trabaje de manera automatica, evitando consumir almacenamiento de repo y local, se procesa y se sube al dw

In [ ]:
import pyarrow.parquet as pq

yellow_taxis = os.listdir(path_yellow)
for file in yellow_taxis:
    table = pq.read_table(path_yellow + '/' + file)
    # Convierte la tabla a un DataFrame de pandas si lo deseas
    df = table.to_pandas()
    # Trabajando con la data

    #Manoseo de archivos si fuese necesario

### Archivos complementarios --> Car Noise

Se extraen datos de ruidos de autos que podrían resultar útiles para las propuestas presentadas

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
# controlador
driver = webdriver.Chrome() 

url = 'https://www.auto-decibel-db.com/desktop.html'
driver.get(url)

# Encontrar la tabla que contiene los datos
table = driver.find_element(By.ID, "resultTable")

# Obtener todas las filas de la tabla (excepto la primera que contiene encabezados)
rows = table.find_elements(By.TAG_NAME, "tr")[1:]

# Inicializar listas para almacenar los datos
data = []

# Iterar a través de las filas y extraer los datos de cada celda
for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    row_data = [cell.text for cell in cells]
    data.append(row_data)

driver.quit()


In [5]:
# Definir los encabezados de las columnas
columns = ["Brand", "Model", "Spec", "Year", "dB at idle", "dB at 55 mph", "dB at 65 mph", "dB at 70 mph", "dB at 75 mph", "dB at 85 mph"]

df = pd.DataFrame(data, columns=columns)
df.head()

,Brand,Model,Spec,Year,dB at idle,dB at 55 mph,dB at 65 mph,dB at 70 mph,dB at 75 mph,dB at 85 mph
0,None,None,None,None,None,None,None,None,None,None
1,Abarth,500,1.4 16v T-Jet,2008,47.3,67.3,70.2,70.4,72.9,75.6
2,Abarth,595,Competizione,2017,49.9,69.1,72.5,72.8,72.2,75.8
3,Acura,MDX,3.5 V6,2013,41.7,59.0,62.0,62.2,64.9,65.9
4,Acura,RDX,3.5 V6,2012,43.0,62.7,66.1,66.3,68.4,69.1


In [9]:
df.drop(index=[0], inplace=True)

In [13]:
df.shape

(1894, 10)

In [14]:
df.to_csv('./Complementary/car_noise.csv', index=False)